In [ ]:
from sklearn.isotonic import isotonic_regression
import cvxpy as cp
import numpy as np
import csv, os
from scipy import stats
from sklearn import metrics
import itertools
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import statistics
from scipy.stats import levene
import seaborn as sns


# Load CSV file into a pandas DataFrame
df = pd.read_csv(r'proxy_score.csv')
df = df.drop_duplicates(['submission_id', 'author_id'])

# Extract the unique authors from the DataFrame
authors = df['author_id'].unique()

# Initialize an empty dictionary to store the results
results_pairup = {}

for i in range(len(authors)):
    for j in range(i + 1, len(authors)):
        author1 = authors[i]
        author2 = authors[j]

        # Extract the submission IDs as sets
        submissions1 = df[df['author_id'] == author1]['submission_id'].tolist()
        submissions2 = df[df['author_id'] == author2]['submission_id'].tolist()

        submissions1 = list(set( submissions1 ))
        submissions2 = list(set( submissions2 ))

        common_submissions = []
        for submission in submissions1:
            if submission in submissions2:
                common_submissions.append(submission)

        # Add the common submissions to the results dictionary
        results_pairup[(author1, author2)] = common_submissions


# All the coauthors that they submit more than 1 paper together
results_coauthor = {}
# Loop through the keys of the dictionary
for key in results_pairup:
    # Check if the value list has more than one element
    if len(results_pairup[key]) > 1:
        key_coauthor = key
        results_coauthor[key_coauthor] = results_pairup[key_coauthor]


results_coauthor = {key: [pair for pair in itertools.combinations(results_coauthor[key], 2)] for key in results_coauthor}

In [4]:

results_coauthor_smallgaps = {}
results_coauthor_largegaps = {}
for author_pair in results_coauthor:
  results_coauthor_smallgaps[author_pair] = []
  results_coauthor_largegaps[author_pair] = []
  for i in range(len(results_coauthor[author_pair])):
    rating_0 = df[ df['submission_id'] == results_coauthor[author_pair][i][0] ]['rating_0422_mean'].tolist()[0]
    rating_1 = df[ df['submission_id'] == results_coauthor[author_pair][i][1] ]['rating_0422_mean'].tolist()[0]
    if abs(rating_0 - rating_1) <= 1:
      results_coauthor_smallgaps[author_pair].append( results_coauthor[author_pair][i] )
    if abs(rating_0 - rating_1) > 1:
      results_coauthor_largegaps[author_pair].append( results_coauthor[author_pair][i] )



smallgaps_total = 0
smallgaps_rank1_disagree = 0
smallgaps_rank1_tie = 0
smallgaps_rank1_agree = 0

# fix each key and each pair of work
for coauthor in results_coauthor_smallgaps:
    smallgaps_total += len(results_coauthor_smallgaps[coauthor])
    if len(results_coauthor_smallgaps[coauthor]) != 0:
      for j in range(len(results_coauthor_smallgaps[coauthor])):
          submission_pair = results_coauthor_smallgaps[coauthor][j]
          submission0 = submission_pair[0]
          submission1 = submission_pair[1]
          author0 = coauthor[0]
          author1 = coauthor[1]
          # Filter the DataFrame to include only the rows corresponding to the key and submission
          rank00 = df[(df['author_id'] == author0) & (df['submission_id'] == submission0)]['rank'].tolist()
          rank01 = df[(df['author_id'] == author0) & (df['submission_id'] == submission1)]['rank'].tolist()
          rank10 = df[(df['author_id'] == author1) & (df['submission_id'] == submission0)]['rank'].tolist()
          rank11 = df[(df['author_id'] == author1) & (df['submission_id'] == submission1)]['rank'].tolist()

          if rank00[0] - rank01[0] == 0 or rank10[0] - rank11[0] == 0:
              smallgaps_rank1_tie += 1
          if (rank00[0] - rank01[0])*(rank10[0] - rank11[0]) < 0:
              smallgaps_rank1_disagree += 1
          if (rank00[0] - rank01[0])*(rank10[0] - rank11[0]) > 0:
              smallgaps_rank1_agree += 1
print("total:", smallgaps_total)
print("fraction of disagree:", smallgaps_rank1_disagree/(smallgaps_rank1_disagree + smallgaps_rank1_agree + smallgaps_rank1_tie) )



largegaps_total = 0
largegaps_rank1_disagree = 0
largegaps_rank1_tie = 0
largegaps_rank1_agree = 0

# fix each key and each pair of work
for coauthor in results_coauthor_largegaps:
    largegaps_total += len(results_coauthor_largegaps[coauthor])
    if len(results_coauthor_largegaps[coauthor]) != 0:
      for j in range(len(results_coauthor_largegaps[coauthor])):
          submission_pair = results_coauthor_largegaps[coauthor][j]
          submission0 = submission_pair[0]
          submission1 = submission_pair[1]
          author0 = coauthor[0]
          author1 = coauthor[1]
          # Filter the DataFrame to include only the rows corresponding to the key and submission
          rank00 = df[(df['author_id'] == author0) & (df['submission_id'] == submission0)]['rank'].tolist()
          rank01 = df[(df['author_id'] == author0) & (df['submission_id'] == submission1)]['rank'].tolist()
          rank10 = df[(df['author_id'] == author1) & (df['submission_id'] == submission0)]['rank'].tolist()
          rank11 = df[(df['author_id'] == author1) & (df['submission_id'] == submission1)]['rank'].tolist()

          if rank00[0] - rank01[0] == 0 or rank10[0] - rank11[0] == 0:
              largegaps_rank1_tie += 1
          if (rank00[0] - rank01[0])*(rank10[0] - rank11[0]) < 0:
              largegaps_rank1_disagree += 1
          if (rank00[0] - rank01[0])*(rank10[0] - rank11[0]) > 0:
              largegaps_rank1_agree += 1
print("total:", largegaps_total)
print("fraction of disagree:", largegaps_rank1_disagree/(largegaps_rank1_disagree + largegaps_rank1_agree + largegaps_rank1_tie) )


total: 433
fraction of disagree: 0.3071593533487298
total: 291
fraction of disagree: 0.2611683848797251
